# S05. 딥 러닝

<h3>합성곱 신경망</h3>

In [1]:
# MINST 데이터 로드
import input_data
mnist = input_data.read_data_sets("./tmp/data/",one_hot=True)
import tensorflow as tf

Extracting ./tmp/data/train-images-idx3-ubyte.gz
Extracting ./tmp/data/train-labels-idx1-ubyte.gz
Extracting ./tmp/data/t10k-images-idx3-ubyte.gz
Extracting ./tmp/data/t10k-labels-idx1-ubyte.gz


In [2]:
# 매개변수 설정
learning_rate  = 0.001
training_iters = 100000
batch_size     = 128
display_step   = 10

In [3]:
# 신경망 관련 매개변수 설정
n_input   = 784   # MNIST 입력 데이터 크기(이미지 모양: 28*28)
n_classes = 10    # MNIST 클래스 개수(0부터 9까지의 숫자 손 글씨)
dropout   = 0.75  # 드롭아웃에서 드롭되지 않을 확률(0에서 1 사이의 값)

In [4]:
# 텐서플로 변수 설정 및 초기화
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])

# 드롭아웃에서 드롭되지 않을 확률(0에서 1 사이의 값)
keep_prob = tf.placeholder(tf.float32)

In [5]:
# 모델 생성
def conv2d(img, w, b):
    return tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(img, w,strides=[1, 1, 1, 1],padding='SAME'),b))

def max_pool(img, k):
    return tf.nn.max_pool(img, ksize=[1, k, k, 1],strides=[1, k, k, 1],padding='SAME')

In [6]:
# 레이어의 가중치 행렬과 편향 행렬 저장
# 5x5 크기의 커널, 1차원의 입력 데이터, 32차원의 출력 데이터
wc1 = tf.Variable(tf.random_normal([5, 5, 1, 32]))
bc1 = tf.Variable(tf.random_normal([32]))

# 5x5 크기의 커널, 32차원의 입력 데이터, 64차원의 출력 데이터
wc2 = tf.Variable(tf.random_normal([5, 5, 32, 64]))
bc2 = tf.Variable(tf.random_normal([64]))

# 완전 연결 레이어, 7*7*64 크기의 입력 데이터, 크기 1024의 출력 데이터
wd1 = tf.Variable(tf.random_normal([7*7*64, 1024]))

# 크기 1024의 입력 데이터, 크기 10의 출력 데이터(입력 데이터에 대한 클래스 개수)
wout = tf.Variable(tf.random_normal([1024, n_classes]))
bd1  = tf.Variable(tf.random_normal([1024]))
bout = tf.Variable(tf.random_normal([n_classes]))

# 모델 생성
_X = tf.reshape(x, shape=[-1, 28, 28, 1])

In [7]:
# 합성곱 레이어
conv1 = conv2d(_X, wc1, bc1)

# 최댓값 풀링(다운 샘플링)
conv1 = max_pool(conv1, k=2)

# 드롭아웃 적용
conv1 = tf.nn.dropout(conv1,keep_prob)

# 합성곱 레이어
conv2 = conv2d(conv1, wc2, bc2)

# 최댓값 풀링(다운 샘플링)
conv2 = max_pool(conv2, k=2)

# 드롭아웃 적용
conv2 = tf.nn.dropout(conv2, keep_prob)

In [8]:
# 완전 연결 레이어
# 완전 연결 레이어 형성을 위해 3차원의 conv2를 1차원으로 변환
dense1 = tf.reshape(conv2, [-1, wd1.get_shape().as_list()[0]])

# ReLU 활성화 함수 적용
dense1 = tf.nn.relu(tf.add(tf.matmul(dense1, wd1),bd1))

# 드롭아웃 적용
dense1 = tf.nn.dropout(dense1, keep_prob)

# 클래스 예측
pred = tf.add(tf.matmul(dense1, wout), bout)

In [9]:
# 비용 함수와 최적화 함수 설정
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# 모델 성능 평가
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# 변수 초기화
init = tf.global_variables_initializer()

In [10]:
# 세션 실행
sess = tf.Session()
sess.run(init)
step = 1

# training_iters에 지정된 횟수만큼 학습 반복
while step * batch_size < training_iters:
    batch_xs, batch_ys = mnist.train.next_batch(batch_size)
    # 모든 배치에 대해 학습 진행
    sess.run(optimizer, feed_dict={x: batch_xs,y: batch_ys,keep_prob: dropout})
    if step % display_step == 0:
        # 배치(batch) 단위의 정확도 평가
        acc = sess.run(accuracy, feed_dict={x: batch_xs,y: batch_ys,keep_prob: 1.})
        # batch 단위의 비용 평가
        loss = sess.run(cost, feed_dict={x: batch_xs,y: batch_ys,keep_prob: 1.})
        print("Iter " + str(step*batch_size) + ",\t Minibatch Loss= " + "{:.6f}".format(loss) + ",\t Training Accuracy= " + "{:.5f}".format(acc))
    step += 1

print("Optimization Finished!")

# 256개의 MNIST 이미지에 대한 성능 평가
print("Testing Accuracy:",sess.run(accuracy,feed_dict={x: mnist.test.images[:256], y: mnist.test.labels[:256],keep_prob: 1.}))

Iter 1280,	 Minibatch Loss= 43246.867188,	 Training Accuracy= 0.11719
Iter 2560,	 Minibatch Loss= 24717.005859,	 Training Accuracy= 0.39062
Iter 3840,	 Minibatch Loss= 13979.078125,	 Training Accuracy= 0.44531
Iter 5120,	 Minibatch Loss= 9683.902344,	 Training Accuracy= 0.61719
Iter 6400,	 Minibatch Loss= 8363.915039,	 Training Accuracy= 0.62500
Iter 7680,	 Minibatch Loss= 6307.967773,	 Training Accuracy= 0.70312
Iter 8960,	 Minibatch Loss= 6082.391602,	 Training Accuracy= 0.70312
Iter 10240,	 Minibatch Loss= 3788.845703,	 Training Accuracy= 0.82031
Iter 11520,	 Minibatch Loss= 4447.407715,	 Training Accuracy= 0.77344
Iter 12800,	 Minibatch Loss= 4019.078613,	 Training Accuracy= 0.77344
Iter 14080,	 Minibatch Loss= 2659.312988,	 Training Accuracy= 0.78125
Iter 15360,	 Minibatch Loss= 2575.833252,	 Training Accuracy= 0.85156
Iter 16640,	 Minibatch Loss= 3779.660645,	 Training Accuracy= 0.79688
Iter 17920,	 Minibatch Loss= 4650.366211,	 Training Accuracy= 0.80469
Iter 19200,	 Minibatch L

0